# Figure S2: Hydro projections by hydrological zones

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr 
import geopandas as gpd

import os
from glob import glob
from oggm import utils
from tqdm import tqdm
from tqdm.notebook import tqdm

import plotly.express as px
import plotly.graph_objects as go
from   plotly.subplots import make_subplots

cl     = px.colors.qualitative.D3
os.chdir('/home/rooda/OGGM_results/')

import warnings
warnings.simplefilter("ignore")

# Ids for each glacier

In [ ]:
RGI6_ids = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI6_v2.shp")
RGI7_ids = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI7_v2.shp")
RGI6_ids = RGI6_ids[RGI6_ids.area_km2 > 1][["RGIId", "Zone"]]
RGI7_ids = RGI7_ids[RGI7_ids.area_km2 > 1]

# RGI6 doesnt have IDs yet 
RGI7_ids = utils.cook_rgidf(RGI7_ids, o1_region='17', o2_region='02', bgndate= RGI7_ids.src_date, 
                            version = "70", assign_column_values= {'Zone' : 'Zone'})

RGI7_ids = RGI7_ids[["RGIId", "Zone"]]
ids = pd.concat([RGI6_ids, RGI7_ids]).set_index("RGIId")

dict_zone = {1:'PPY', 2:'PCA', 3:'NPI-E', 4:'NPI-W', 5:'SPI-N', 6:'SPI-C', 7:'SPI-S', 8:'GCN', 9:'CDI'}
ids = ids.replace({"Zone": dict_zone})

In [ ]:
def preprocess(ds): # remove unnecessary variables and coordinates
    return ds.drop_vars(['hydro_year', 'hydro_month', 'calendar_year', 'calendar_month'])

def hydro_variables(ds): # calculate total_runoff, melt_on_glacier and smb
    ds["total_runoff"] = ((ds.melt_off_glacier + ds.melt_on_glacier + ds.liq_prcp_off_glacier + ds.liq_prcp_on_glacier)*1e-3)/(365*86400) # m3/s
    ds["melt_on_glacier"] = ((ds.melt_on_glacier)*1e-3)/(365*86400) # m3
    ds["ratio"] = ds["melt_on_glacier"] / ds["total_runoff"]
    return ds[variables]

def postprocessing(ds, scenario): # clean dataframe
    ds = ds.to_dataframe()
    ds["scenario"] = scenario
    ds = ds.set_index("scenario", append=True)
    ds = ds.reorder_levels(['scenario', 'rgi_id', 'time'])
    return ds

In [ ]:
# variables to analize
variables        = ['total_runoff', 'melt_on_glacier', 'ratio']
scenarios        = ["ct_random", "ssp126","ssp245","ssp370","ssp585"]

In [ ]:
# historical period
all_combs = glob("/home/rooda/OGGM_results/new/*/run_outputs_*.nc", recursive = True)
all_opts   = xr.open_mfdataset(all_combs, combine='nested', concat_dim="options", chunks="auto", parallel=True, preprocess=preprocess)

# assing zone to each glacier and aggregate the result 
ids_subset = ids[ids.index.isin(all_opts.rgi_id.to_pandas().tolist())]
all_opts   = all_opts.assign_coords(rgi_id = ids_subset.Zone.tolist())
all_opts   = all_opts.groupby('rgi_id').sum()
all_opts   = all_opts.chunk("auto")
all_opts   = hydro_variables(all_opts)
all_opts   = all_opts.isel(time = slice(0, -1))

# standard desviation
dataset_var_hist = all_opts.std(dim="options")
dataset_var_hist = postprocessing(dataset_var_hist, "historical")

# mean
dataset_mean_hist = all_opts.mean(dim="options")
dataset_mean_hist = postprocessing(dataset_mean_hist, "historical")

In [ ]:
dataset_mean = []
dataset_var  = []

for scenario in tqdm(scenarios): 
    all_combs = glob("/home/rooda/OGGM_results/new/*/run_output_*"+ scenario +"*.nc", recursive = True)
    all_opts   = xr.open_mfdataset(all_combs, combine='nested', concat_dim="options", chunks="auto", parallel=True, preprocess=preprocess)

    if scenario == "ct_random":
        all_opts["time"] = all_opts["time"] + 2020
    
    # assing zone to each glacier and aggregate the result 
    ids_subset = ids[ids.index.isin(all_opts.rgi_id.to_pandas().tolist())]
    all_opts   = all_opts.assign_coords(rgi_id = ids_subset.Zone.tolist())
    all_opts   = all_opts.groupby('rgi_id').sum()
    all_opts   = all_opts.chunk("auto")
    all_opts   = hydro_variables(all_opts)
    all_opts   = all_opts.isel(time = slice(0, -1))
    
    # standard desviation
    all_opts_var = all_opts.std(dim="options")
    all_opts_var = postprocessing(all_opts_var, scenario)
    dataset_var.append(all_opts_var)
    
    # mean
    all_opts_mean = all_opts.mean(dim="options")
    all_opts_mean = postprocessing(all_opts_mean, scenario)
    dataset_mean.append(all_opts_mean)
    
dataset_mean = pd.concat(dataset_mean)
dataset_mean = pd.concat([dataset_mean_hist, dataset_mean]).reset_index()

dataset_var  = pd.concat(dataset_var)
dataset_var  = pd.concat([dataset_var_hist, dataset_var]).reset_index()

In [ ]:
dict_replace = {"scenario": {"historical":'Historical', "ct_random":'Commitment run', "ssp126":'SSP 1-2.6', "ssp245":'SSP 2-4.5', "ssp370":'SSP 3-7.0', "ssp585":'SSP 5-8.5'}}

dataset_mean = dataset_mean.replace(dict_replace)
dataset_var  = dataset_var.replace(dict_replace)

In [ ]:
scenarios   = ["Historical", "Commitment run", "SSP 1-2.6", "SSP 2-4.5", "SSP 3-7.0", "SSP 5-8.5"]
scen_colors = {"Historical":"rgba(0, 0, 0, 0.8)", "Commitment run":"rgba(0, 0, 0, 0.5)", "SSP 1-2.6":cl[0], "SSP 2-4.5":cl[2], "SSP 3-7.0":cl[1], "SSP 5-8.5":cl[3]}
shaded_colors = {"Historical":"rgba(0, 0, 0, 0.15)", "SSP 1-2.6": "rgba(31, 119, 180,0.15)", "SSP 5-8.5":"rgba(214, 39, 40, 0.15)"}

basins_id  = ['PPY', 'PCA','NPI-E','NPI-W','SPI-N', 'SPI-C', 'SPI-S', 'GCN', 'CDI']

fig    = make_subplots(rows=9, cols=3, horizontal_spacing = 0.05, vertical_spacing = 0.01, 
                       shared_xaxes= True, shared_yaxes= False, row_titles = basins_id,
                       subplot_titles= ["Total runoff (m<sup>3</sup> s<sup>-1</sup>)", "Melt on glacier (m<sup>3</sup> s<sup>-1</sup>)", "Ratio"])

for x in range(0,9):
    for y in range(0,3):
        for t in range(0,6):
             
            # time series for each subplot and scenario
            time_series_id    = dataset_mean[dataset_mean.rgi_id == basins_id[x]][dataset_mean.scenario == scenarios[t]]
            time_series_sd_id = dataset_var[dataset_var.rgi_id == basins_id[x]][dataset_var.scenario == scenarios[t]]    
            
            if x==0 and y==0: # legend only for first plot
        
                fig.add_trace(go.Scatter(x=time_series_id.time, y=time_series_id.iloc[:,y+3], 
                                         mode='lines', name= scenarios[t], 
                                         line=dict(color=scen_colors[scenarios[t]], width = 1.5), showlegend=True, legendgroup=t), row=x+1, col=y+1)
                
                if t == 0 or t == 2 or t >= 5: # uncertainty: +-1 sd
                    fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=(time_series_id.iloc[:,y+3]+time_series_sd_id.iloc[:,y+3]), 
                                             line=dict(width=0), fillcolor=shaded_colors[scenarios[t]], showlegend=False, legendgroup='g1'), row=x+1, col=y+1)
                    fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=(time_series_id.iloc[:,y+3]-time_series_sd_id.iloc[:,y+3]), 
                                             line=dict(width=0), fillcolor=shaded_colors[scenarios[t]], fill='tonexty', showlegend=False, legendgroup=t), row=x+1, col=y+1)

            else:
                # mean value
                fig.add_trace(go.Scatter(x=time_series_id.time, y=time_series_id.iloc[:,y+3], mode='lines', name= scenarios[t], 
                                         line=dict(color=scen_colors[scenarios[t]], width = 1.5), showlegend=False, legendgroup=t), row=x+1, col=y+1)

                if t == 0 or t == 2 or t >= 5: # uncertainty: +-1 sd# uncertainty: +-1 sd
                    fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=(time_series_id.iloc[:,y+3]+time_series_sd_id.iloc[:,y+3]), 
                                             line=dict(width=0), fillcolor=shaded_colors[scenarios[t]], showlegend=False, legendgroup=t), row=x+1, col=y+1)
                    fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=(time_series_id.iloc[:,y+3]-time_series_sd_id.iloc[:,y+3]), 
                                             line=dict(width=0), fillcolor=shaded_colors[scenarios[t]], fill='tonexty', showlegend=False, legendgroup=t), row=x+1, col=y+1)

#  some tweaks
for x in range(0,9):
    for y in range(0,3):                
        fig.update_yaxes(tickformat=".0%", dtick = 0.2, range = [0.1, 0.75], row=x+1, col=3)

        
fig.update_yaxes(dtick = 30 , row=1, col=1) 
fig.update_yaxes(dtick = 100, row=3, col=1) 
fig.update_yaxes(dtick = 500, row=6, col=1)  
fig.update_yaxes(dtick = 200, row=7, col=1)  
fig.update_yaxes(dtick = 150, row=9, col=1)  
fig.update_yaxes(dtick = 300, row=5, col=2) 
fig.update_yaxes(dtick = 500, row=6, col=2) 


fig.update_yaxes(ticks="outside", griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", showline = True, linecolor = 'black', linewidth = 0.2, mirror=True, tickangle = -90)
fig.update_xaxes(ticks="outside", griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", showline = True, linecolor = 'black', linewidth = 0.2, mirror=True, dtick = 20)
fig.update_layout(legend=dict(yanchor="top", y=-0.02, xanchor="left", x=0.15, orientation="h", bgcolor = 'rgba(0,0,0,0.0)'))
fig.update_layout(height=1200, width=1050, template = "seaborn", margin = dict(l=20, r=20, b=20, t=20), hovermode = False)

# save figure 
fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_S2_hydro_projections.png", scale=4)
fig.show()